In [1]:
# Project setup: create project folder and write minimal JSON data for 3 numeral systems
import os, json
PROJECT_DIR = "/content/global_numeral_app"
os.makedirs(PROJECT_DIR, exist_ok=True)

roman = {
  "name": "Roman",
  "type": "additive-subtractive",
  "symbols": [
    {"symbol":"M","value":1000},{"symbol":"CM","value":900},{"symbol":"D","value":500},
    {"symbol":"CD","value":400},{"symbol":"C","value":100},{"symbol":"XC","value":90},
    {"symbol":"L","value":50},{"symbol":"XL","value":40},{"symbol":"X","value":10},
    {"symbol":"IX","value":9},{"symbol":"V","value":5},{"symbol":"IV","value":4},{"symbol":"I","value":1}
  ],
  "notes": "Modern Roman numerals, uses subtractive pairs (IV, IX, XL, XC, CD, CM)."
}

mayan = {
  "name": "Mayan (simplified)",
  "type": "positional-vigesimal",
  "base": 20,
  "digit_representation": {
    "dot": 1,
    "bar": 5,
    "shell": 0
  },
  "notes": "Simplified textual encoding: each vigesimal digit written as bars (B) and dots (o), with 'S' for shell (0). Lowest place first in compact token form."
}

chinese = {
  "name": "Chinese (modern spoken-digit mapping)",
  "type": "morpho-decimal",
  "mapping": {
    "0": "零", "1": "一", "2": "二", "3": "三", "4": "四",
    "5": "五", "6": "六", "7": "七", "8": "八", "9": "九",
    "10": "十", "100": "百", "1000": "千", "10000": "万"
  },
  "notes": "This entry captures a simple spoken/character mapping and place-value markers for demonstration and conversion examples."
}

# write files
for name, data in [("roman.json", roman), ("mayan.json", mayan), ("chinese.json", chinese)]:
    path = os.path.join(PROJECT_DIR, name)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
print("Wrote JSON files to", PROJECT_DIR)
print("Files:", os.listdir(PROJECT_DIR))


Wrote JSON files to /content/global_numeral_app
Files: ['chinese.json', 'roman.json', 'mayan.json']


In [2]:
# Roman converter (clean, tested) — Arabic <-> Roman
import json, os, re
PROJECT_DIR = "/content/global_numeral_app"

# load roman.json (created earlier)
with open(os.path.join(PROJECT_DIR, "roman.json"), "r", encoding="utf-8") as f:
    roman_data = json.load(f)

# Ordered symbol list (high to low)
SYMBOLS = [(s["symbol"], s["value"]) for s in roman_data["symbols"]]

def arabic_to_roman(n):
    """
    Convert a positive integer n -> Roman numeral string.
    Returns: (roman_string, steps_list)
    """
    if not isinstance(n, int) or n <= 0:
        raise ValueError("Input must be a positive integer (>=1).")
    remaining = n
    parts = []
    steps = []
    for sym, val in SYMBOLS:
        if remaining <= 0:
            break
        count = remaining // val
        if count > 0:
            # add symbol count times (works because SYMBOLS contains subtractive combos like CM before C)
            parts.append(sym * count)
            steps.append(f"Use {count} × {sym} (value {val}) -> contributes {count * val}")
            remaining -= count * val
    roman_str = "".join(parts)
    steps.append(f"Final Roman numeral: {roman_str} (remaining {remaining})")
    return roman_str, steps

def roman_to_arabic(s):
    """
    Convert Roman numeral string s -> integer.
    Returns: (value, steps_list)
    Accepts standard modern forms using subtractive notation (IV, IX, XL, XC, CD, CM).
    """
    if not isinstance(s, str) or not s.strip():
        raise ValueError("Input must be a non-empty Roman numeral string.")
    s = s.upper().strip()
    steps = []
    total = 0
    i = 0

    # Build quick lookup dicts
    two_letter = {sym: val for sym, val in SYMBOLS if len(sym) == 2}
    one_letter = {sym: val for sym, val in SYMBOLS if len(sym) == 1}

    while i < len(s):
        # Try two-letter match first (subtractive forms)
        if i + 1 < len(s) and s[i:i+2] in two_letter:
            sym = s[i:i+2]
            val = two_letter[sym]
            steps.append(f"Matched {sym} -> {val}")
            total += val
            i += 2
        elif s[i] in one_letter:
            sym = s[i]
            val = one_letter[sym]
            steps.append(f"Matched {sym} -> {val}")
            total += val
            i += 1
        else:
            # invalid symbol encountered
            raise ValueError(f"Invalid Roman numeral sequence at position {i}: '{s[i:]}'")
    steps.append(f"Computed total = {total}")
    return total, steps

# --- Tests / Examples ---
tests_arabic = [1, 4, 9, 14, 44, 99, 2019, 3999]
print("Arabic -> Roman (examples):")
for n in tests_arabic:
    r, steps = arabic_to_roman(n)
    print(f"{n} -> {r} | steps: {steps}")

print("\nRoman -> Arabic (examples):")
tests_roman = ["I", "IV", "IX", "XIV", "XLIV", "XCIX", "MMXIX", "MMMCMXCIX"]
for s in tests_roman:
    val, steps = roman_to_arabic(s)
    print(f"{s} -> {val} | steps: {steps}")

# Round-trip quick check for a range
failed = []
for n in range(1, 500):  # test 1..499
    r, _ = arabic_to_roman(n)
    back, _ = roman_to_arabic(r)
    if back != n:
        failed.append((n, r, back))
if failed:
    print(f"\nRound-trip mismatches (sample): {failed[:5]}")
else:
    print("\nRound-trip check passed for 1..499.")


Arabic -> Roman (examples):
1 -> I | steps: ['Use 1 × I (value 1) -> contributes 1', 'Final Roman numeral: I (remaining 0)']
4 -> IV | steps: ['Use 1 × IV (value 4) -> contributes 4', 'Final Roman numeral: IV (remaining 0)']
9 -> IX | steps: ['Use 1 × IX (value 9) -> contributes 9', 'Final Roman numeral: IX (remaining 0)']
14 -> XIV | steps: ['Use 1 × X (value 10) -> contributes 10', 'Use 1 × IV (value 4) -> contributes 4', 'Final Roman numeral: XIV (remaining 0)']
44 -> XLIV | steps: ['Use 1 × XL (value 40) -> contributes 40', 'Use 1 × IV (value 4) -> contributes 4', 'Final Roman numeral: XLIV (remaining 0)']
99 -> XCIX | steps: ['Use 1 × XC (value 90) -> contributes 90', 'Use 1 × IX (value 9) -> contributes 9', 'Final Roman numeral: XCIX (remaining 0)']
2019 -> MMXIX | steps: ['Use 2 × M (value 1000) -> contributes 2000', 'Use 1 × X (value 10) -> contributes 10', 'Use 1 × IX (value 9) -> contributes 9', 'Final Roman numeral: MMXIX (remaining 0)']
3999 -> MMMCMXCIX | steps: ['Use 3 × 

In [3]:
# Mayan converter (simplified vigesimal positional) — Arabic <-> compact textual Mayan
import os, json, math
PROJECT_DIR = "/content/global_numeral_app"

# Representation convention:
# - Each vigesimal digit (0..19) encoded as:
#     'S' -> shell (0)
#     repeated 'B' characters -> bars (each bar = 5)
#     repeated 'o' characters -> dots (each dot = 1)
#   e.g., 13 -> BBooo (2 bars -> 10, 3 dots -> 3)
# - Full number compact token: tokens joined by '-' from lowest place to highest place.
#   e.g., 25 -> lowest digit 5 ('B'), next digit 1 ('o') -> "B-o"
# - Visual presentation often shown highest->lowest for readability.

def digit_to_mayan_token(d):
    """Convert single digit 0..19 to (token, visual_desc)."""
    if not (0 <= d <= 19):
        raise ValueError("Mayan digit must be in 0..19")
    if d == 0:
        return "S", "shell (0)"
    bars = d // 5
    dots = d % 5
    token = "B" * bars + "o" * dots
    visual = []
    if bars:
        visual.append(f"bars:{bars}")
    if dots:
        visual.append(f"dots:{dots}")
    return token, ", ".join(visual)

def arabic_to_mayan(n):
    """
    Convert non-negative integer n -> (compact_token_low_to_high, visual_high_to_low, steps)
    Uses pure base-20 positional system.
    """
    if not isinstance(n, int) or n < 0:
        raise ValueError("Arabic input must be a non-negative integer.")
    steps = []
    if n == 0:
        tok, vis = digit_to_mayan_token(0)
        steps.append("Input is 0 -> shell")
        return tok, [vis], steps

    digits = []
    place = 0
    remaining = n
    while remaining > 0:
        d = remaining % 20
        digits.append(d)  # lowest-first
        steps.append(f"Place {place} (20^{place}): digit = {d}")
        remaining //= 20
        place += 1

    tokens = []
    visuals = []
    for d in digits:
        tok, vis = digit_to_mayan_token(d)
        tokens.append(tok)
        visuals.append(vis)

    compact = "-".join(tokens)            # lowest->highest
    visual_high_to_low = list(reversed(visuals))
    steps.append(f"Digits (lowest->highest): {digits}")
    steps.append(f"Compact token (lowest->highest): {compact}")
    return compact, visual_high_to_low, steps

def mayan_token_to_digit(tok):
    """Parse a single token (e.g., 'BBoo' or 'S') into integer 0..19."""
    if not isinstance(tok, str):
        raise ValueError("Token must be a string.")
    tok = tok.strip()
    if tok == "" or tok == "S":
        return 0
    if set(tok) - set("Bo"):
        raise ValueError(f"Invalid characters in Mayan digit token: '{tok}'")
    bars = tok.count("B")
    dots = tok.count("o")
    val = bars * 5 + dots
    if val < 0 or val > 19:
        raise ValueError(f"Decoded digit out of range from token '{tok}' -> {val}")
    return val

def mayan_to_arabic(compact):
    """
    Convert compact mayan string (lowest->highest tokens separated by '-') to Arabic integer.
    Returns (value, steps)
    Example: 'B-o' -> 25 (5 + 1*20)
    """
    if not isinstance(compact, str) or compact.strip() == "":
        raise ValueError("Mayan compact string must be a non-empty string.")
    tokens = [t.strip() for t in compact.split("-")]
    steps = []
    total = 0
    for place, tok in enumerate(tokens):
        val = mayan_token_to_digit(tok)
        contrib = val * (20 ** place)
        steps.append(f"Token '{tok}' at place {place} (20^{place}) -> digit {val} -> contributes {contrib}")
        total += contrib
    steps.append(f"Computed total = {total}")
    return total, steps

def pretty_mayan_visual(compact):
    """
    Produce a readable visual for a compact Mayan token.
    Returns a string showing highest->lowest with 'bars/dots/shell' notes per place.
    """
    tokens = [t.strip() for t in compact.split("-")]
    visuals = []
    for tok in reversed(tokens):  # highest -> lowest
        if tok == "" or tok == "S":
            visuals.append("shell (0)")
        else:
            bars = tok.count("B")
            dots = tok.count("o")
            parts = []
            if bars:
                parts.append(f"{bars} bar(s)")
            if dots:
                parts.append(f"{dots} dot(s)")
            visuals.append(", ".join(parts))
    return " | ".join(visuals)

# -------------------------
# Tests and round-trip checks
# -------------------------
print("Mayan converter tests (examples):")
examples = [0, 1, 4, 5, 6, 13, 19, 20, 25, 399, 400, 2019, 12345]
for n in examples:
    comp, vis, steps = arabic_to_mayan(n)
    back, back_steps = mayan_to_arabic(comp)
    print(f"{n} -> compact: '{comp}' | visual (high->low): {vis} | round-trip: {back}")
    # quick sanity check
    if back != n:
        print("  >>> Round-trip mismatch for", n)

# Round-trip check over a range
failed = []
for n in range(0, 2000):
    comp, vis, steps = arabic_to_mayan(n)
    back, _ = mayan_to_arabic(comp)
    if back != n:
        failed.append((n, comp, back))
        if len(failed) > 10:
            break

if not failed:
    print("\nRound-trip check passed for 0..1999.")
else:
    print("\nRound-trip mismatches (sample):", failed[:5])


Mayan converter tests (examples):
0 -> compact: 'S' | visual (high->low): ['shell (0)'] | round-trip: 0
1 -> compact: 'o' | visual (high->low): ['dots:1'] | round-trip: 1
4 -> compact: 'oooo' | visual (high->low): ['dots:4'] | round-trip: 4
5 -> compact: 'B' | visual (high->low): ['bars:1'] | round-trip: 5
6 -> compact: 'Bo' | visual (high->low): ['bars:1, dots:1'] | round-trip: 6
13 -> compact: 'BBooo' | visual (high->low): ['bars:2, dots:3'] | round-trip: 13
19 -> compact: 'BBBoooo' | visual (high->low): ['bars:3, dots:4'] | round-trip: 19
20 -> compact: 'S-o' | visual (high->low): ['dots:1', 'shell (0)'] | round-trip: 20
25 -> compact: 'B-o' | visual (high->low): ['dots:1', 'bars:1'] | round-trip: 25
399 -> compact: 'BBBoooo-BBBoooo' | visual (high->low): ['bars:3, dots:4', 'bars:3, dots:4'] | round-trip: 399
400 -> compact: 'S-S-o' | visual (high->low): ['dots:1', 'shell (0)', 'shell (0)'] | round-trip: 400
2019 -> compact: 'BBBoooo-S-B' | visual (high->low): ['bars:1', 'shell (0)'

In [4]:
# Chinese converter (modern spoken-digit mapping) — Arabic <-> simple Chinese characters
import os, json, re
PROJECT_DIR = "/content/global_numeral_app"

# load chinese.json (created earlier)
with open(os.path.join(PROJECT_DIR, "chinese.json"), "r", encoding="utf-8") as f:
    chinese_data = json.load(f)

# mapping from the JSON file (characters for digits and place markers)
char_map = chinese_data["mapping"]
digit_char = {int(k): v for k, v in char_map.items() if k.isdigit()}
# place markers as strings (e.g., 10 -> '十', 100 -> '百', 1000 -> '千', 10000 -> '万')
place_markers = {int(k): v for k, v in char_map.items() if int(k) >= 10}

# Helper: Arabic -> spoken Chinese (characters)
def arabic_to_chinese(n):
    """
    Convert integer n (0 <= n <= 99999) to a modern Chinese character representation.
    Returns: (chinese_string, steps_list)
    Notes:
      - This is a simple, readable representation. It uses 万/千/百/十 positions and zero-handling.
      - Examples:
         0 -> '零'
         10 -> '十'
         20 -> '二十'
         105 -> '一百零五'
         10001 -> '一万零一'
    """
    if not isinstance(n, int) or n < 0 or n > 99999:
        raise ValueError("This converter supports integers 0..99999.")
    steps = []
    if n == 0:
        return digit_char[0], ["Input is zero -> 零"]

    parts = []
    remaining = n

    wan = remaining // 10000
    if wan:
        parts.append((wan, 10000))
    remaining %= 10000

    qian = remaining // 1000
    if qian:
        parts.append((qian, 1000))
    remaining %= 1000

    bai = remaining // 100
    if bai:
        parts.append((bai, 100))
    remaining %= 100

    shi = remaining // 10
    ge = remaining % 10

    # Build string with correct zero handling
    s = ""
    # handle wan block
    if wan:
        s += digit_char[wan] + place_markers[10000]
        steps.append(f"万 place: {wan} -> {digit_char[wan]}{place_markers[10000]}")
        if (qian == 0 and bai == 0 and shi == 0 and ge != 0):
            # special: 10001 -> 一万零一
            s += digit_char[0]
            steps.append("Inserted 零 between 万 and units because intermediate places are zero")
    # handle qian/bai/shi/ge within 0..9999
    if qian:
        s += digit_char[qian] + place_markers[1000]
        steps.append(f"千 place: {qian} -> {digit_char[qian]}{place_markers[1000]}")
    else:
        if s and (bai or shi or ge):
            # previous higher place existed and current is zero -> maybe need 零 later
            pass

    if bai:
        # if preceding was zero (qian==0) and some higher existed, ensure we insert 零
        if s and s[-1] != digit_char[0] and (not qian) and (wan):
            # If wan existed and qian==0 and we haven't already added 零 for continuity, insert zero when needed
            if bai:
                # don't always add here — rules vary; keep conservative: no extra zero if qian==0 but bai exists
                pass
        s += digit_char[bai] + place_markers[100]
        steps.append(f"百 place: {bai} -> {digit_char[bai]}{place_markers[100]}")
    else:
        if s and (shi or ge) and (qian != 0):
            # if qian existed and bai is zero but lower places non-zero, insert 零
            s += digit_char[0]
            steps.append("Inserted 零 because a middle place (百) was zero between nonzero places")

    if shi:
        # handle special case: 10..19 usually written as '十' or '十一' etc.
        if shi == 1 and not (wan or qian or bai):
            s += place_markers[10]  # '十'
            steps.append("Ten's place is 1 and it's the highest non-zero -> use '十'")
        else:
            s += digit_char[shi] + place_markers[10]
            steps.append(f"十 place: {shi} -> {digit_char[shi]}{place_markers[10]}")
    else:
        if s and (ge != 0) and (not (bai or qian or wan)):
            # numbers like 5 -> '五' (we won't add zero here)
            pass
        elif s and (ge != 0) and (not shi) and ( (bai or qian or wan) ):
            # if there are higher places and tens is zero but units non-zero, need 零
            s += digit_char[0]
            steps.append("Inserted 零 because 十 place is zero but lower units exist")

    if ge:
        s += digit_char[ge]
        steps.append(f"Units: {ge} -> {digit_char[ge]}")

    # final cleanup: remove any accidental duplicate 零
    s = re.sub(r'零+', '零', s)
    s = s.rstrip('零')
    steps.append(f"Final Chinese string: {s}")
    return s, steps

# Helper: Chinese characters -> Arabic (parser for forms produced above)
def chinese_to_arabic(chs):
    """
    Parse a Chinese numeric string (in the simplified modern forms produced by arabic_to_chinese)
    back into an integer. Supports up to 万 (10000).
    Returns (value, steps)
    """
    if not isinstance(chs, str) or not chs.strip():
        raise ValueError("Input must be a non-empty Chinese numeric string.")
    s = chs.strip()
    steps = []
    # quick map from char->digit for digits 0..9
    rev_digit = {v: k for k, v in digit_char.items()}
    rev_place = {v: k for k, v in place_markers.items()}  # e.g., '十' -> 10
    total = 0
    # handle 万 separately
    if '万' in s:
        parts = s.split('万')
        left = parts[0]
        right = parts[1] if len(parts) > 1 else ""
        left_val = 0
        # left could be single-digit like '一' or more complex (but we built only 1..9)
        if left:
            if left in rev_digit:
                left_val = rev_digit[left]
            else:
                # try parse simple left like '十二' -> 12 (rare in our outputs)
                lv = 0
                # naive parse: handle hundreds/tens if present
                # fallback: map digits sequentially
                for ch in left:
                    if ch in rev_digit:
                        lv = lv * 10 + rev_digit[ch]
                left_val = lv
        total += left_val * 10000
        steps.append(f"Found 万 block: '{left}' -> {left_val} × 10000 -> {left_val*10000}")
        s = right  # continue parsing right side

    # parse thousands/hundreds/tens/units in the remainder using place markers
    current = 0
    temp = 0
    i = 0
    # iterate characters and interpret digits and markers
    j = 0
    value = 0
    # Map simpler parsing: for each place marker, extract preceding digit (if any)
    # thousands
    if '千' in s:
        idx = s.index('千')
        ch = s[:idx]
        digit = rev_digit.get(ch, 1) if ch else 1
        value += digit * 1000
        steps.append(f"千 block: '{ch}' -> {digit} × 1000 -> {digit*1000}")
        s = s[idx+1:]
    if '百' in s:
        idx = s.index('百')
        ch = s[:idx]
        digit = rev_digit.get(ch, 1) if ch else 1
        value += digit * 100
        steps.append(f"百 block: '{ch}' -> {digit} × 100 -> {digit*100}")
        s = s[idx+1:]
    if '十' in s:
        idx = s.index('十')
        ch = s[:idx]
        if ch == '':
            digit = 1
        else:
            digit = rev_digit.get(ch, 1)
        value += digit * 10
        steps.append(f"十 block: '{ch}' -> {digit} × 10 -> {digit*10}")
        s = s[idx+1:]
    # remaining are units or '零' markers
    s = s.replace('零', '')
    if s:
        # remaining characters should be digits
        for ch in s:
            if ch in rev_digit:
                value += rev_digit[ch]
                steps.append(f"Units part: '{ch}' -> {rev_digit[ch]}")
    total += value
    steps.append(f"Computed total = {total}")
    return total, steps

# -------------------------
# Tests and round-trip checks
# -------------------------
print("Chinese converter examples:")
examples = [0, 5, 10, 11, 20, 105, 110, 101, 1001, 10001, 12345, 99999]
for n in examples:
    if n > 99999:
        continue
    chs, steps = arabic_to_chinese(n)
    back, back_steps = chinese_to_arabic(chs)
    print(f"{n} -> '{chs}' | round-trip -> {back}")
    if back != n:
        print("  >>> Round-trip mismatch for", n)
        print("    encode steps:", steps)
        print("    parse steps:", back_steps)

# A few explicit prints for clarity
print("\nExample: 105 ->", arabic_to_chinese(105))
print("Example: 10001 ->", arabic_to_chinese(10001))
print("Example: 12345 ->", arabic_to_chinese(12345))


Chinese converter examples:
0 -> '零' | round-trip -> 0
5 -> '五' | round-trip -> 5
10 -> '十' | round-trip -> 10
11 -> '十一' | round-trip -> 11
20 -> '二十' | round-trip -> 20
105 -> '一百零五' | round-trip -> 105
110 -> '一百一十' | round-trip -> 110
101 -> '一百零一' | round-trip -> 101
1001 -> '一千零一' | round-trip -> 1001
10001 -> '一万零一' | round-trip -> 10001
12345 -> '一万二千三百四十五' | round-trip -> 12345
99999 -> '九万九千九百九十九' | round-trip -> 99999

Example: 105 -> ('一百零五', ['百 place: 1 -> 一百', 'Inserted 零 because 十 place is zero but lower units exist', 'Units: 5 -> 五', 'Final Chinese string: 一百零五'])
Example: 10001 -> ('一万零一', ['万 place: 1 -> 一万', 'Inserted 零 between 万 and units because intermediate places are zero', 'Inserted 零 because 十 place is zero but lower units exist', 'Units: 1 -> 一', 'Final Chinese string: 一万零一'])
Example: 12345 -> ('一万二千三百四十五', ['万 place: 1 -> 一万', '千 place: 2 -> 二千', '百 place: 3 -> 三百', '十 place: 4 -> 四十', 'Units: 5 -> 五', 'Final Chinese string: 一万二千三百四十五'])


In [5]:
# Unified converter engine: use Roman, Mayan, Chinese converters already defined
import json, os
PROJECT_DIR = "/content/global_numeral_app"

# Assumes these functions are already present in the notebook:
# arabic_to_roman, roman_to_arabic
# arabic_to_mayan, mayan_to_arabic, pretty_mayan_visual
# arabic_to_chinese, chinese_to_arabic

def convert(value, system):
    """
    Unified convert API.
      - value: int (Arabic) -> converts to 'system'
               str -> interprets as system representation and converts to Arabic
      - system: one of 'roman', 'mayan', 'chinese' (case-insensitive)
    Returns dict:
      { 'system': system, 'input': value, 'direction': 'arabic_to_<system>' or '<system>_to_arabic',
        'result': ..., 'human_readable': ..., 'steps': [...] }
    """
    sys_lower = system.strip().lower()
    out = {"system": sys_lower, "input": value}
    # Arabic -> system
    if isinstance(value, int):
        out["direction"] = f"arabic_to_{sys_lower}"
        if sys_lower == "roman":
            res, steps = arabic_to_roman(value)
            out.update({"result": res, "steps": steps, "human_readable": f"{value} in Roman is {res}."})
        elif sys_lower == "mayan":
            res, visual, steps = arabic_to_mayan(value)
            out.update({"result": res, "steps": steps, "human_readable": f"{value} in simplified Mayan (lowest->highest compact token) is {res}. Visual (high->low): {visual}."})
        elif sys_lower == "chinese":
            res, steps = arabic_to_chinese(value)
            out.update({"result": res, "steps": steps, "human_readable": f"{value} in modern Chinese characters is {res}."})
        else:
            raise ValueError("Unsupported system: " + system)
        return out

    # system -> Arabic (string input)
    elif isinstance(value, str):
        out["direction"] = f"{sys_lower}_to_arabic"
        if sys_lower == "roman":
            val, steps = roman_to_arabic(value)
            out.update({"result": val, "steps": steps, "human_readable": f"{value} (Roman) -> {val} (Arabic)."})
        elif sys_lower == "mayan":
            val, steps = mayan_to_arabic(value)
            out.update({"result": val, "steps": steps, "human_readable": f"{value} (Mayan compact) -> {val} (Arabic). Visual: {pretty_mayan_visual(value)}"})
        elif sys_lower == "chinese":
            val, steps = chinese_to_arabic(value)
            out.update({"result": val, "steps": steps, "human_readable": f"{value} (Chinese) -> {val} (Arabic)."})
        else:
            raise ValueError("Unsupported system: " + system)
        return out
    else:
        raise ValueError("Input `value` must be int (Arabic) or str (system representation).")

def pretty_print_conversion(res):
    print("=== Conversion ===")
    print("System:", res["system"])
    print("Input :", res["input"])
    print("Direction:", res["direction"])
    print("Result :", res["result"])
    print("\nHuman readable:", res["human_readable"])
    print("\nSteps:")
    for s in res["steps"]:
        print(" -", s)
    print("===================\n")

# ---- Demo examples ----
demo_cases = [
    (2019, "Roman"),
    ("MMXIX", "Roman"),
    (25, "Mayan"),
    ("B-o", "Mayan"),
    (12345, "Chinese"),
    ("一万二千三百四十五", "Chinese")
]

for val, sys in demo_cases:
    try:
        out = convert(val, sys)
        pretty_print_conversion(out)
    except Exception as e:
        print("Error for", val, sys, "=>", e)


=== Conversion ===
System: roman
Input : 2019
Direction: arabic_to_roman
Result : MMXIX

Human readable: 2019 in Roman is MMXIX.

Steps:
 - Use 2 × M (value 1000) -> contributes 2000
 - Use 1 × X (value 10) -> contributes 10
 - Use 1 × IX (value 9) -> contributes 9
 - Final Roman numeral: MMXIX (remaining 0)

=== Conversion ===
System: roman
Input : MMXIX
Direction: roman_to_arabic
Result : 2019

Human readable: MMXIX (Roman) -> 2019 (Arabic).

Steps:
 - Matched M -> 1000
 - Matched M -> 1000
 - Matched X -> 10
 - Matched IX -> 9
 - Computed total = 2019

=== Conversion ===
System: mayan
Input : 25
Direction: arabic_to_mayan
Result : B-o

Human readable: 25 in simplified Mayan (lowest->highest compact token) is B-o. Visual (high->low): ['dots:1', 'bars:1'].

Steps:
 - Place 0 (20^0): digit = 5
 - Place 1 (20^1): digit = 1
 - Digits (lowest->highest): [5, 1]
 - Compact token (lowest->highest): B-o

=== Conversion ===
System: mayan
Input : B-o
Direction: mayan_to_arabic
Result : 25

Huma

In [6]:
# Practice Zone: puzzle generator + auto-checker + export
import random, json, os
PROJECT_DIR = "/content/global_numeral_app"
EXPORT_PATH = os.path.join(PROJECT_DIR, "practice_puzzles.json")
random.seed(2)

# Uses existing converters in the notebook:
# - arabic_to_roman, roman_to_arabic
# - arabic_to_mayan, mayan_to_arabic, pretty_mayan_visual
# - arabic_to_chinese, chinese_to_arabic

# --- Puzzle generators (deterministic-ish, reproducible with seed) ---
def gen_pattern_identification(system="Roman", length=4, start=None, step=None):
    """Generate arithmetic progression and present its representations in `system`."""
    start = start if start is not None else random.randint(2, 30)
    step = step if step is not None else random.randint(1, 10)
    nums = [start + i*step for i in range(length)]
    next_num = start + length*step
    if system.lower() == "roman":
        seq = [arabic_to_roman(n)[0] for n in nums]
        next_repr = arabic_to_roman(next_num)[0]
    elif system.lower() == "mayan":
        seq = [arabic_to_mayan(n)[0] for n in nums]
        next_repr = arabic_to_mayan(next_num)[0]
    elif system.lower() == "chinese":
        seq = [arabic_to_chinese(n)[0] for n in nums]
        next_repr = arabic_to_chinese(next_num)[0]
    else:
        raise ValueError("Unsupported system")
    statement = f"Identify the pattern in these {system} numerals and give the next one:\nSequence: {', '.join(seq)}"
    solution = next_repr
    explanation = f"Underlying Arabic sequence: {nums} (step = {step}), next = {next_num} -> {solution}"
    return {"type":"Pattern Identification", "system":system, "statement":statement, "solution":solution, "explanation":explanation}

def gen_construction_challenge(system="Roman", target=None):
    """Ask to construct representation of `target` in `system`."""
    target = target if target is not None else random.randint(7, 400)
    if system.lower() == "roman":
        sol = arabic_to_roman(target)[0]
    elif system.lower() == "mayan":
        sol = arabic_to_mayan(target)[0]
    elif system.lower() == "chinese":
        sol = arabic_to_chinese(target)[0]
    else:
        raise ValueError("Unsupported system")
    statement = f"Construct the representation of the number {target} in the {system} numeral system."
    explanation = f"The correct representation is {sol}."
    return {"type":"Construction", "system":system, "target":target, "statement":statement, "solution":sol, "explanation":explanation}

def gen_rule_inference(system="Roman"):
    """Give a few examples (Arabic -> system) and ask to decode a target."""
    nums = random.sample(range(5, 200), 3)
    if system.lower() == "roman":
        reps = [arabic_to_roman(n)[0] for n in nums]
    elif system.lower() == "mayan":
        reps = [arabic_to_mayan(n)[0] for n in nums]
    elif system.lower() == "chinese":
        reps = [arabic_to_chinese(n)[0] for n in nums]
    else:
        raise ValueError("Unsupported system")
    target = random.randint(20, 300)
    if system.lower() == "roman":
        target_sol = arabic_to_roman(target)[0]
    elif system.lower() == "mayan":
        target_sol = arabic_to_mayan(target)[0]
    else:
        target_sol = arabic_to_chinese(target)[0]
    examples = "\n".join([f"{a} → {b}" for a, b in zip(nums, reps)])
    statement = f"Examples from unknown numeral system ({system}):\n{examples}\nUsing these examples, decode the number {target}."
    explanation = f"By applying the mapping shown in the examples, {target} corresponds to {target_sol}."
    return {"type":"Rule Inference", "system":system, "examples":list(zip(nums,reps)), "target":target, "statement":statement, "solution":target_sol, "explanation":explanation}

# --- High-level generator that supports selection ---
def generate_puzzle(kind=None, system=None):
    """Generate one puzzle. kind can be 'pattern','construction','inference' or None for random.
       system can be 'Roman','Mayan','Chinese' or None for random.
    """
    systems = ["Roman","Mayan","Chinese"]
    system = system if system is not None else random.choice(systems)
    if kind is None:
        kind = random.choice(["pattern","construction","inference"])
    if kind == "pattern":
        return gen_pattern_identification(system=system)
    if kind == "construction":
        return gen_construction_challenge(system=system)
    if kind == "inference":
        return gen_rule_inference(system=system)
    raise ValueError("Unknown kind")

# --- Auto-checker ---
def check_answer(puzzle, answer):
    """Check user's answer against puzzle['solution'].
       Accepts string answers (for system forms). Returns dict {correct:bool, expected:..., note:...}
    """
    expected = str(puzzle["solution"]).strip()
    ans = str(answer).strip()
    # For Chinese, normalize zeros/whitespace
    if puzzle["system"].lower() == "chinese":
        expected = expected.replace(" ", "")
        ans = ans.replace(" ", "")
    # For Roman: uppercase
    if puzzle["system"].lower() == "roman":
        expected = expected.upper()
        ans = ans.upper()
    # For Mayan: normalize token formatting (remove spaces)
    if puzzle["system"].lower() == "mayan":
        expected = expected.replace(" ", "")
        ans = ans.replace(" ", "")
    correct = (ans == expected)
    note = "Exact match required for this auto-checker. You can expand to accept equivalent forms."
    return {"correct": correct, "expected": expected, "given": ans, "note": note}

# --- Utilities: batch-generate, export, and demo ---
def generate_batch(n=10, seed=None):
    if seed is not None:
        random.seed(seed)
    puzzles = [generate_puzzle() for _ in range(n)]
    return puzzles

def export_puzzles(puzzles, path=EXPORT_PATH):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(puzzles, f, ensure_ascii=False, indent=2)
    return path

# --- Demo: generate 6 puzzles and show statements + solutions (for instructor/demo) ---
demo = generate_batch(6, seed=42)
for i, p in enumerate(demo, 1):
    print(f"Puzzle {i} — Type: {p['type']} ({p['system']})")
    print("Statement:")
    print(p['statement'])
    print("Solution (for demo):", p['solution'])
    print("Explanation:", p['explanation'])
    print("-"*60)

# Save demo to JSON for submission
exported = export_puzzles(demo)
print("\nExported demo puzzles to:", exported)


Puzzle 1 — Type: Pattern Identification (Chinese)
Statement:
Identify the pattern in these Chinese numerals and give the next one:
Sequence: 二, 七, 十二, 十七
Solution (for demo): 二十二
Explanation: Underlying Arabic sequence: [2, 7, 12, 17] (step = 5), next = 22 -> 二十二
------------------------------------------------------------
Puzzle 2 — Type: Pattern Identification (Roman)
Statement:
Identify the pattern in these Roman numerals and give the next one:
Sequence: VI, VIII, X, XII
Solution (for demo): XIV
Explanation: Underlying Arabic sequence: [6, 8, 10, 12] (step = 2), next = 14 -> XIV
------------------------------------------------------------
Puzzle 3 — Type: Rule Inference (Chinese)
Statement:
Examples from unknown numeral system (Chinese):
144 → 一百四十四
27 → 二十七
156 → 一百五十六
Using these examples, decode the number 236.
Solution (for demo): 二百三十六
Explanation: By applying the mapping shown in the examples, 236 corresponds to 二百三十六.
----------------------------------------------------------